In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import TimeDistributed, GlobalAveragePooling1D, GlobalAveragePooling2D, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling1D
#from keras.layers.embeddings import Embedding
from keras.layers import Dropout, Flatten, Bidirectional, Dense, Activation, TimeDistributed
from keras.models import Model, Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from string import ascii_lowercase
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models import doc2vec
from gensim.models import KeyedVectors
import itertools, nltk, snowballstemmer, re

LabeledSentence = doc2vec.LabeledSentence

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return shuffled

In [3]:
data = pd.read_csv('deceptive-opinion-spam-corpus.zip', compression='zip', header=0, sep=',', quotechar='"')

In [4]:
data['polarity'] = np.where(data['polarity']=='positive', 1, 0)
data['deceptive'] = np.where(data['deceptive']=='truthful', 1, 0)

In [5]:
def create_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return [1,1]
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return [1,0]
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return [0,1]
    else:
        return [0,0]
    
def specific_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return "TRUE_POSITIVE"
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return "FALSE_POSITIVE"
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return "TRUE_NEGATIVE"
    else:
        return "FALSE_NEGATIVE"

data['final_class'] = data.apply(create_class, axis=1)
data['given_class'] = data.apply(specific_class, axis=1)

In [6]:
Y = data['final_class']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [7]:
textData = pd.DataFrame(list(data['text'])) # each row is one document; the raw text of the document should be in the 'text_data' column

In [8]:
# initialize stemmer
stemmer = snowballstemmer.EnglishStemmer()

# grab stopword list, extend it a bit, and then turn it into a set for later
stop = stopwords.words('english')
stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across','among','beside','however','yet','within']+list(ascii_lowercase))
stoplist = stemmer.stemWords(stop)
stoplist = set(stoplist)
stop = set(sorted(stop + list(stoplist))) 

In [9]:
# remove characters and stoplist words, then generate dictionary of unique words
textData[0].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)
wordlist = filter(None, " ".join(list(set(list(itertools.chain(*textData[0].str.split(' ')))))).split(" "))
data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in textData[0].str.lower().str.split(' ')]

In [10]:
# remove all words that don't occur at least 5 times and then stem the resulting docs
minimum_count = 1
str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*data['stemmed_text_data'].str.split(' '))))).items()),columns=['word','count'])
low_frequency_words = set(str_frequencies[str_frequencies['count'] < minimum_count]['word'])
data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in low_frequency_words, line))) for line in data['stemmed_text_data'].str.split(' ')]
data['stemmed_text_data'] = [" ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', next_text).split(' '))) for next_text in data['stemmed_text_data']]    

In [11]:
lmtzr = WordNetLemmatizer()
w = re.compile("\w+",re.I)

def label_sentences(df, input_point):
    labeled_sentences = []
    list_sen = []
    for index, datapoint in df.iterrows():
        tokenized_words = re.findall(w,datapoint[input_point].lower())
        labeled_sentences.append(LabeledSentence(words=tokenized_words, tags=['SENT_%s' %index]))
        list_sen.append(tokenized_words)
    return labeled_sentences, list_sen

def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(min_count=1, window=9, size=512, sample=1e-4, negative=5, workers=7)
    model.build_vocab(labeled_sentences)
    pretrained_weights = model.wv.syn0
    vocab_size, embedding_size = pretrained_weights.shape
    model.train(labeled_sentences, total_examples=vocab_size, epochs=400)
    
    return model

In [12]:
textData = data['stemmed_text_data'].to_frame().reset_index()
sen, corpus = label_sentences(textData, 'stemmed_text_data')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':


In [13]:
doc2vec_model = train_doc2vec_model(sen)

/usr/local/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  app.launch_new_instance()


In [ ]:
doc2vec_model.save("doc2vec_model_opinion_corpus.d2v")

In [14]:
doc2vec_model = Doc2Vec.load("doc2vec_model_opinion_corpus.d2v") 

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf1 = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, ngram_range=(1,1))
result_train1 = tfidf1.fit_transform(corpus)

tfidf2 = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, ngram_range=(1,2))
result_train2 = tfidf2.fit_transform(corpus)

tfidf3 = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, ngram_range=(1,3))
result_train3 = tfidf3.fit_transform(corpus)

svd = TruncatedSVD(n_components=512, n_iter=40, random_state=34)
tfidf_data1 = svd.fit_transform(result_train1)
tfidf_data2 = svd.fit_transform(result_train2)
tfidf_data3 = svd.fit_transform(result_train3)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import spacy

nlp = spacy.load('en')
temp_textData = pd.DataFrame(list(data['text']))

overall_pos_tags_tokens = []
overall_pos = []
overall_tokens = []
overall_dep = []

for i in range(1600):
    doc = nlp(temp_textData[0][i])
    given_pos_tags_tokens = []
    given_pos = []
    given_tokens = []
    given_dep = []
    for token in doc:
        output = "%s_%s" % (token.pos_, token.tag_)
        given_pos_tags_tokens.append(output)
        given_pos.append(token.pos_)
        given_tokens.append(token.tag_)
        given_dep.append(token.dep_)
        
    overall_pos_tags_tokens.append(given_pos_tags_tokens)
    overall_pos.append(given_pos)
    overall_tokens.append(given_tokens)
    overall_dep.append(given_dep)


In [17]:
from sklearn.preprocessing import MinMaxScaler

count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
pos_tags_data = count.fit_transform(overall_pos_tags_tokens).todense()
pos_data = count.fit_transform(overall_pos).todense()
tokens_data = count.fit_transform(overall_tokens).todense()
dep_data = count.fit_transform(overall_dep).todense()
min_max_scaler = MinMaxScaler()
normalized_pos_tags_data = min_max_scaler.fit_transform(pos_tags_data)
normalized_pos_data = min_max_scaler.fit_transform(pos_data)
normalized_tokens_data = min_max_scaler.fit_transform(tokens_data)
normalized_dep_data = min_max_scaler.fit_transform(dep_data)

final_pos_tags_data = np.zeros(shape=(1600, 512)).astype(np.float32)
final_pos_data = np.zeros(shape=(1600, 512)).astype(np.float32)
final_tokens_data = np.zeros(shape=(1600, 512)).astype(np.float32)
final_dep_data = np.zeros(shape=(1600, 512)).astype(np.float32)
final_pos_tags_data[:normalized_pos_tags_data.shape[0],:normalized_pos_tags_data.shape[1]] = normalized_pos_tags_data
final_pos_data[:normalized_pos_data.shape[0],:normalized_pos_data.shape[1]] = normalized_pos_data
final_tokens_data[:normalized_tokens_data.shape[0],:normalized_tokens_data.shape[1]] = normalized_tokens_data
final_dep_data[:normalized_dep_data.shape[0],:normalized_dep_data.shape[1]] = normalized_dep_data

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [18]:
maxlength = []
for i in range(0,len(sen)):
    maxlength.append(len(sen[i][0]))
    
print(max(maxlength))   

370


In [19]:
def vectorize_comments(df,d2v_model):
    y = []
    comments = []
    for i in range(0,df.shape[0]):
        label = 'SENT_%s' %i
        comments.append(d2v_model.docvecs[label])
    df['vectorized_comments'] = comments
    
    return df

textData = vectorize_comments(textData,doc2vec_model)
print (textData.head(2))

   index                                  stemmed_text_data  \
0      0  stay night getaway famili thursday tripl aaa r...   
1      1  tripl rate upgrad view room less $ includ brea...   

                                 vectorized_comments  
0  [-0.23642673, -0.48943135, 0.29437515, 0.27929...  
1  [0.050735053, -0.962265, 0.084685884, 0.161688...  


In [20]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [21]:
from nltk.corpus import stopwords

glove_data = np.zeros(shape=(1600, 800, 512)).astype(np.float32)
temp_textData = data['text'].to_frame().reset_index()
sen2, corpus2 = label_sentences(temp_textData, 'text')
stop_words = set(stopwords.words('english'))
test_word = np.zeros(512).astype(np.float32)
final_matrix = np.zeros(512).astype(np.float32)
final_sizes = []

count = True

for i in range(1600):
    for j in sen2[i][0]:
        if j in embeddings_index and j not in stop_words:
            test_word[:300] = embeddings_index[j]
            if count == True:
                final_matrix = test_word
                count = False
            else:
                final_matrix = np.vstack((final_matrix, test_word))
            
    final_sizes.append(final_matrix.shape[0])
    final_matrix = np.zeros(512).astype(np.float32)
    glove_data[i,:final_matrix.shape[0],:] = final_matrix
    count = True
    
print(max(final_sizes))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':


389


In [22]:
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
X_train, X_test, y_train, y_test = cross_validation.train_test_split(textData["vectorized_comments"].T.tolist(), 
                                                                     dummy_y, 
                                                                     test_size=0.1, 
                                                                     random_state=56)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [23]:
X = np.array(textData["vectorized_comments"].T.tolist()).reshape((1,1600,512))
y = np.array(dummy_y).reshape((1600,4))
X_train2 = np.array(X_train).reshape((1,1440,512))
y_train2 = np.array(y_train).reshape((1,1440,4))
X_test2 = np.array(X_test).reshape((1,160,512))
y_test2 = np.array(y_test).reshape((1,160,4))

In [24]:
from sklearn.model_selection import StratifiedKFold
Xtemp = textData["vectorized_comments"].T.tolist()
ytemp = data['given_class']
training_indices = []
testing_indices = []

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(Xtemp, ytemp)

for train_index, test_index in skf.split(Xtemp, ytemp):
    training_indices.append(train_index)
    testing_indices.append(test_index)

In [54]:
def extractTrainingAndTestingData(givenIndex):
    X_train3 = np.zeros(shape=(1440, max(maxlength)+10+379, 512)).astype(np.float32)
    Y_train3 = np.zeros(shape=(1440, 4)).astype(np.float32)
    X_test3 = np.zeros(shape=(160, max(maxlength)+10+379, 512)).astype(np.float32)
    Y_test3 = np.zeros(shape=(160, 4)).astype(np.float32)

    empty_word = np.zeros(512).astype(np.float32)

    count_i = 0
    for i in training_indices[givenIndex]:
        len1 = len(sen[i][0])
        average_vector1 = np.zeros(512).astype(np.float32)
        average_vector2 = np.zeros(512).astype(np.float32)
        average_vector3 = np.zeros(512).astype(np.float32)
        for j in range(max(maxlength)+10+379):
            if j < len1:
                X_train3[count_i,j,:] = doc2vec_model[sen[i][0][j]]
                average_vector1 += result_train1[i, tfidf1.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]
                average_vector2 += result_train2[i, tfidf2.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]
                average_vector3 += result_train3[i, tfidf3.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]
            elif j >= len1 and j < len1 + 379:
                X_train3[count_i,j,:] = glove_data[i, j-len1, :]
            elif j == len1 + 379:
                X_train3[count_i,j,:] = tfidf_data1[i]
            elif j == len1 + 380:
                X_train3[count_i,j,:] = tfidf_data2[i]
            elif j == len1+381:
                X_train3[count_i,j,:] = tfidf_data3[i]
            elif j == len1+382:
                X_train3[count_i,j,:] = average_vector1
            elif j == len1+383:
                X_train3[count_i,j,:] = average_vector2
            elif j == len1+384:
                X_train3[count_i,j,:] = average_vector3
            elif j == len1+385:
                X_train3[count_i,j,:] = final_pos_tags_data[i] 
            elif j == len1+386:
                X_train3[count_i,j,:] = final_pos_data[i]
            elif j == len1+387:
                X_train3[count_i,j,:] = final_tokens_data[i]
            elif j == len1+388:
                X_train3[count_i,j,:] = final_dep_data[i]
            else:
                X_train3[count_i,j,:] = empty_word

        Y_train3[count_i,:] = dummy_y[i]
        count_i += 1


    count_i = 0
    for i in testing_indices[givenIndex]:
        len1 = len(sen[i][0])
        average_vector1 = np.zeros(512).astype(np.float32)
        average_vector2 = np.zeros(512).astype(np.float32)
        average_vector3 = np.zeros(512).astype(np.float32)
        for j in range(max(maxlength)+10+379):
            if j < len1:
                X_test3[count_i,j,:] = doc2vec_model[sen[i][0][j]]
                average_vector1 += result_train1[i, tfidf1.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]
                average_vector2 += result_train2[i, tfidf2.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]  
                average_vector3 += result_train3[i, tfidf3.vocabulary_[sen[i][0][j]]] * doc2vec_model[sen[i][0][j]]
            elif j >= len1 and j < len1 + 379:
                X_test3[count_i,j,:] = glove_data[i, j-len1, :]
            elif j == len1 + 379:
                X_test3[count_i,j,:] = tfidf_data1[i]
            elif j == len1 + 380:
                X_test3[count_i,j,:] = tfidf_data2[i]
            elif j == len1+381:
                X_test3[count_i,j,:] = tfidf_data3[i]
            elif j == len1+382:
                X_test3[count_i,j,:] = average_vector1
            elif j == len1+383:
                X_test3[count_i,j,:] = average_vector2
            elif j == len1+384:
                X_test3[count_i,j,:] = average_vector3
            elif j == len1+385:
                X_test3[count_i,j,:] = final_pos_tags_data[i]
            elif j == len1+386:
                X_test3[count_i,j,:] = final_pos_data[i]
            elif j == len1+387:
                X_test3[count_i,j,:] = final_tokens_data[i]
            elif j == len1+388:
                X_test3[count_i,j,:] = final_dep_data[i]
            else:
                X_test3[count_i,j,:] = empty_word

        Y_test3[count_i,:] = dummy_y[i]
        count_i += 1
        
    return X_train3, X_test3, Y_train3, Y_test3
    

In [76]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu', input_shape=(max(maxlength)+10+379,512)))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
#model.add(Conv1D(filters=32, kernel_size=7, padding='same', activation='relu'))
#model.add(Dropout(0.25))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))
#model.add(Conv1D(filters=32, kernel_size=9, padding='same', activation='relu'))
#model.add(Dropout(0.25))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))
#model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
#model.add(Dropout(0.25))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))

#model.add(Bidirectional(LSTM(50, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(50, dropout=0.25, recurrent_dropout=0.2)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 759, 128)          327808    
_________________________________________________________________
dropout_55 (Dropout)         (None, 759, 128)          0         
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 379, 128)          0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 379, 128)          0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 100)               71600     
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 404       
Total params: 399,812
Trainable params: 399,812
Non-trainable params: 0
_________________________________________________________________
None

In [77]:
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint 

final_accuracies = []

for i in range(10):
    filename = 'weights.best.from_scratch%s.hdf5' % i
    checkpointer = ModelCheckpoint(filepath=filename, verbose=1, save_best_only=True)
    X_train3, X_test3, Y_train3, Y_test3 = extractTrainingAndTestingData(i)
    model.fit(X_train3, Y_train3, epochs=40, batch_size=512, callbacks=[checkpointer], validation_data=(X_test3, Y_test3))
    model.load_weights(filename)
    predicted = np.rint(model.predict(X_test3))
    final_accuracies.append(accuracy_score(Y_test3, predicted))
    print(accuracy_score(Y_test3, predicted))

Train on 1440 samples, validate on 160 samples
Epoch 1/40
1440/1440 [==============================] - 29s 20ms/step - loss: 0.5496 - acc: 0.7500 - val_loss: 0.5045 - val_acc: 0.7500

Epoch 00001: val_loss improved from inf to 0.50446, saving model to weights.best.from_scratch0.hdf5
Epoch 2/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.4971 - acc: 0.7517 - val_loss: 0.4342 - val_acc: 0.7781

Epoch 00002: val_loss improved from 0.50446 to 0.43420, saving model to weights.best.from_scratch0.hdf5
Epoch 3/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.4361 - acc: 0.7814 - val_loss: 0.3733 - val_acc: 0.8500

Epoch 00003: val_loss improved from 0.43420 to 0.37330, saving model to weights.best.from_scratch0.hdf5
Epoch 4/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.3858 - acc: 0.8278 - val_loss: 0.3064 - val_acc: 0.8906

Epoch 00004: val_loss improved from 0.37330 to 0.30644, saving model to weights.best.from_scratch

1440/1440 [==============================] - 23s 16ms/step - loss: 0.1149 - acc: 0.9585 - val_loss: 0.0468 - val_acc: 0.9812

Epoch 00002: val_loss improved from 0.05096 to 0.04684, saving model to weights.best.from_scratch1.hdf5
Epoch 3/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.1037 - acc: 0.9632 - val_loss: 0.0468 - val_acc: 0.9781

Epoch 00003: val_loss improved from 0.04684 to 0.04679, saving model to weights.best.from_scratch1.hdf5
Epoch 4/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0965 - acc: 0.9682 - val_loss: 0.0606 - val_acc: 0.9719

Epoch 00004: val_loss did not improve
Epoch 5/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0993 - acc: 0.9642 - val_loss: 0.0489 - val_acc: 0.9812

Epoch 00005: val_loss did not improve
Epoch 6/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0860 - acc: 0.9720 - val_loss: 0.0435 - val_acc: 0.9844

Epoch 00006: val_loss improved from 0.0467

1440/1440 [==============================] - 24s 16ms/step - loss: 0.0469 - acc: 0.9854 - val_loss: 0.0607 - val_acc: 0.9750

Epoch 00007: val_loss did not improve
Epoch 8/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0420 - acc: 0.9875 - val_loss: 0.0776 - val_acc: 0.9688

Epoch 00008: val_loss did not improve
Epoch 9/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.0334 - acc: 0.9918 - val_loss: 0.0669 - val_acc: 0.9719

Epoch 00009: val_loss did not improve
Epoch 10/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0368 - acc: 0.9908 - val_loss: 0.1076 - val_acc: 0.9594

Epoch 00010: val_loss did not improve
Epoch 11/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0355 - acc: 0.9894 - val_loss: 0.0882 - val_acc: 0.9688

Epoch 00011: val_loss did not improve
Epoch 12/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0294 - acc: 0.9925 - val_loss: 0.1000 - val_acc: 0.96


Epoch 00012: val_loss did not improve
Epoch 13/40
1440/1440 [==============================] - 56s 39ms/step - loss: 0.0248 - acc: 0.9951 - val_loss: 0.0486 - val_acc: 0.9812

Epoch 00013: val_loss did not improve
Epoch 14/40
1440/1440 [==============================] - 79s 55ms/step - loss: 0.0216 - acc: 0.9957 - val_loss: 0.0557 - val_acc: 0.9812

Epoch 00014: val_loss did not improve
Epoch 15/40
1440/1440 [==============================] - 3448s 2s/step - loss: 0.0234 - acc: 0.9939 - val_loss: 0.0637 - val_acc: 0.9750

Epoch 00015: val_loss did not improve
Epoch 16/40
1440/1440 [==============================] - 495s 344ms/step - loss: 0.0219 - acc: 0.9950 - val_loss: 0.0845 - val_acc: 0.9625

Epoch 00016: val_loss did not improve
Epoch 17/40
1440/1440 [==============================] - 964s 669ms/step - loss: 0.0176 - acc: 0.9965 - val_loss: 0.0704 - val_acc: 0.9750

Epoch 00017: val_loss did not improve
Epoch 18/40
1440/1440 [==============================] - 22s 15ms/step - loss


Epoch 00018: val_loss did not improve
Epoch 19/40
1440/1440 [==============================] - 22s 16ms/step - loss: 0.0153 - acc: 0.9972 - val_loss: 0.0405 - val_acc: 0.9812

Epoch 00019: val_loss did not improve
Epoch 20/40
1440/1440 [==============================] - 26s 18ms/step - loss: 0.0136 - acc: 0.9967 - val_loss: 0.0624 - val_acc: 0.9766

Epoch 00020: val_loss did not improve
Epoch 21/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0167 - acc: 0.9965 - val_loss: 0.0482 - val_acc: 0.9859

Epoch 00021: val_loss did not improve
Epoch 22/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.0156 - acc: 0.9965 - val_loss: 0.0467 - val_acc: 0.9891

Epoch 00022: val_loss did not improve
Epoch 23/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0170 - acc: 0.9960 - val_loss: 0.0572 - val_acc: 0.9781

Epoch 00023: val_loss did not improve
Epoch 24/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.


Epoch 00024: val_loss did not improve
Epoch 25/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0105 - acc: 0.9979 - val_loss: 0.0528 - val_acc: 0.9797

Epoch 00025: val_loss did not improve
Epoch 26/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0128 - acc: 0.9972 - val_loss: 0.0450 - val_acc: 0.9812

Epoch 00026: val_loss did not improve
Epoch 27/40
1440/1440 [==============================] - 25s 17ms/step - loss: 0.0144 - acc: 0.9957 - val_loss: 0.0528 - val_acc: 0.9734

Epoch 00027: val_loss did not improve
Epoch 28/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0110 - acc: 0.9976 - val_loss: 0.0451 - val_acc: 0.9812

Epoch 00028: val_loss did not improve
Epoch 29/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0116 - acc: 0.9962 - val_loss: 0.0423 - val_acc: 0.9828

Epoch 00029: val_loss did not improve
Epoch 30/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.


Epoch 00030: val_loss did not improve
Epoch 31/40
1440/1440 [==============================] - 25s 17ms/step - loss: 0.0070 - acc: 0.9993 - val_loss: 0.0925 - val_acc: 0.9719

Epoch 00031: val_loss did not improve
Epoch 32/40
1440/1440 [==============================] - 25s 17ms/step - loss: 0.0079 - acc: 0.9990 - val_loss: 0.0824 - val_acc: 0.9734

Epoch 00032: val_loss did not improve
Epoch 33/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0096 - acc: 0.9972 - val_loss: 0.0835 - val_acc: 0.9719

Epoch 00033: val_loss did not improve
Epoch 34/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.0926 - val_acc: 0.9719

Epoch 00034: val_loss did not improve
Epoch 35/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0070 - acc: 0.9990 - val_loss: 0.0792 - val_acc: 0.9688

Epoch 00035: val_loss did not improve
Epoch 36/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.

1440/1440 [==============================] - 25s 17ms/step - loss: 0.0077 - acc: 0.9984 - val_loss: 0.1037 - val_acc: 0.9734

Epoch 00036: val_loss did not improve
Epoch 37/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0091 - acc: 0.9974 - val_loss: 0.1137 - val_acc: 0.9688

Epoch 00037: val_loss did not improve
Epoch 38/40
1440/1440 [==============================] - 25s 17ms/step - loss: 0.0121 - acc: 0.9967 - val_loss: 0.1088 - val_acc: 0.9688

Epoch 00038: val_loss did not improve
Epoch 39/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.0090 - acc: 0.9979 - val_loss: 0.1155 - val_acc: 0.9688

Epoch 00039: val_loss did not improve
Epoch 40/40
1440/1440 [==============================] - 26s 18ms/step - loss: 0.0096 - acc: 0.9983 - val_loss: 0.1155 - val_acc: 0.9688

Epoch 00040: val_loss did not improve
0.9875
Train on 1440 samples, validate on 160 samples
Epoch 1/40
1440/1440 [==============================] - 24s 17ms/step - loss: 

1440/1440 [==============================] - 23s 16ms/step - loss: 0.0461 - acc: 0.9872 - val_loss: 0.0196 - val_acc: 0.9953

Epoch 00001: val_loss improved from inf to 0.01960, saving model to weights.best.from_scratch9.hdf5
Epoch 2/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0415 - acc: 0.9889 - val_loss: 0.0203 - val_acc: 0.9969

Epoch 00002: val_loss did not improve
Epoch 3/40
1440/1440 [==============================] - 24s 17ms/step - loss: 0.0453 - acc: 0.9863 - val_loss: 0.0276 - val_acc: 0.9922

Epoch 00003: val_loss did not improve
Epoch 4/40
1440/1440 [==============================] - 26s 18ms/step - loss: 0.0432 - acc: 0.9868 - val_loss: 0.0215 - val_acc: 0.9953

Epoch 00004: val_loss did not improve
Epoch 5/40
1440/1440 [==============================] - 23s 16ms/step - loss: 0.0391 - acc: 0.9889 - val_loss: 0.0249 - val_acc: 0.9906

Epoch 00005: val_loss did not improve
Epoch 6/40
1440/1440 [==============================] - 23s 16ms/step - lo

In [79]:
print(sum(final_accuracies) / len(final_accuracies))

0.9712500000000002
